In [1]:
import numpy as np
import matplotlib.pyplot as plt

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# import tensorflow as tf
# from tensorflow.keras.losses import CategoricalCrossentropy
import torch
import torch.nn.functional as F
from System import FICAR

# Fuzzy Loss Function

In [9]:
from FuzzyLoss import FuzzyLoss

y_true = torch.tensor([4,  1], dtype=torch.int64)
y_pred = torch.tensor([[0 , .7 , 0 ,0 ,  .3], [0, 0.9, 0, 0.1, 0]])

print('DF-CELoss : ', F.cross_entropy(y_pred, y_true).numpy())
print('DF-CELoss : ', torch.nn.CrossEntropyLoss()(y_pred, y_true).numpy())
print('FuzzyLoss : ', FuzzyLoss(gamma=0.)(y_pred, y_true).numpy())

DF-CELoss :  1.2661572
DF-CELoss :  1.2661572
FuzzyLoss :  1.2661572


Let's test it with a simple VGG16 + Cifar10 dataset

In [7]:
from tensorflow.keras import datasets
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
#Normalize the pixel valuesº
train_images, test_images = train_images / 255.0, test_images / 255.0

170500096/170498071 [==============================] - 15s 0us/step


In [8]:
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.float32)
train_labels.

In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(input_shape = (32, 32, 3), include_top = False, weights = 'imagenet')
base_model.trainable=False
model=base_model.output
model=Flatten()(model)
model=Dense(4096, activation='relu')(model)
model=Dropout(rate=0.5)(model)
model=Dense(4096, activation='relu')(model)
model=Dropout(rate=0.5)(model)
model=Dense(10, activation='softmax')(model)
model=Model(inputs=base_model.inputs, outputs=model)

In [23]:
model.compile(optimizer='adam', loss=FuzzyLoss(gamma=2), metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

Epoch 1/5
1563/1563 [==============================] - 27s 17ms/step - loss: nan - accuracy: 0.0999 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/5
1563/1563 [==============================] - 28s 18ms/step - loss: nan - accuracy: 0.1014 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/5
1563/1563 [==============================] - 28s 18ms/step - loss: nan - accuracy: 0.1004 - val_loss: nan - val_accuracy: 0.1000
Epoch 4/5
1563/1563 [==============================] - 28s 18ms/step - loss: nan - accuracy: 0.0992 - val_loss: nan - val_accuracy: 0.1000
Epoch 5/5
1563/1563 [==============================] - 29s 18ms/step - loss: nan - accuracy: 0.0989 - val_loss: nan - val_accuracy: 0.1000


In [24]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

313/313 - 3s - loss: nan - accuracy: 0.1000


In [21]:
test_loss, test_acc

(nan, 0.10000000149011612)

# System

In [6]:
system = FICAR(n_classes=4, class_names=['male_w', 'female_w', 'male_b', 'female_b'])
system.train("dummyDL")

P(0,1)-> Fit with pair ('male_w', 'female_w')
P(0,2)-> Fit with pair ('male_w', 'male_b')
P(0,3)-> Fit with pair ('male_w', 'female_b')
P(1,2)-> Fit with pair ('female_w', 'male_b')
P(1,3)-> Fit with pair ('female_w', 'female_b')
P(2,3)-> Fit with pair ('male_b', 'female_b')


In [7]:
y_pred = system.predict('dummyIns')
print(system.decisions)
print(y_pred)

[[0.         0.3120504  0.95452845 0.81355755]
 [0.6879496  0.         0.24934205 0.20867684]
 [0.04547155 0.75065795 0.         0.10901914]
 [0.18644245 0.79132316 0.89098086 0.        ]]
male_b
